In [1]:
import PyPDF2 

try:
    from urllib import urlretrieve as urlretrieve
except ImportError:
    from urllib.request import urlretrieve as urlretrieve    

import os

#import matplotlib as plt
import pylab as pl

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Reference: https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

In [5]:
url = []
url.append('https://github.com/fedhere/UInotebooks/raw/master/slides2018/UI1_PUI2018.pdf')
url.append('https://github.com/fedhere/UInotebooks/raw/master/slides2018/UI2_PUI2018.pdf')
url.append('https://github.com/fedhere/UInotebooks/raw/master/slides2018/UI3_PUI2018.pdf')
url.append('https://github.com/fedhere/UInotebooks/raw/master/slides2018/UI4_PUI2018.pdf')
url.append('https://github.com/fedhere/UInotebooks/raw/master/slides2018/UI5_PUI2018.pdf')
url.append('https://github.com/fedhere/UInotebooks/raw/master/slides2018/UI7_PUI2018.pdf')


In [6]:
### This function downloads the pdf file, moves it into the PUIdata and reads it into a dataframe
def getPdfUrl(url,pdfFile):
    #csvFile = 'file.csv'
    urlretrieve(url, pdfFile)
    os.system("mv " + pdfFile + ' ' + os.getenv("PUIDATA"))

In [7]:
## https://www.geeksforgeeks.org/working-with-pdf-files-in-python/

def extractText(pdfFilePath):
    # creating a pdf file object 
    pdfFileObj = open(pdfFilePath, 'rb') 

    # creating a pdf reader object 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  
    # printing number of pages in pdf file 
    n = pdfReader.numPages

    text = ""
    for i in range(n):
        # creating a page object 
        pageObj = pdfReader.getPage(i)
    
        # extracting text from page 
        text += pageObj.extractText()
 
    # closing the pdf file object 
    pdfFileObj.close() 
    
    return text

In [8]:
fileNames = ['UI'+str(i) for i in range(1,7)]

In [9]:
texts = []
AllText = ''
for i in range(len(url)):
    getPdfUrl(url[i],fileNames[i])
    pdfFilePath = os.getenv('PUIDATA') + '/' + fileNames[i]
    txt = extractText(pdfFilePath)
    texts.append(txt)
    AllText += txt

In [10]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [12]:
# clean document
doc = AllText
tokens = clean_doc(doc)
print(tokens[:5])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['urban', 'informatics', 'fall', 'dr', 'federica']
Total Tokens: 11277
Unique Tokens: 1966


In [13]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 11226


In [14]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [15]:
# save sequences to file
out_filename = 'pui_sequences.txt'
save_doc(sequences, out_filename)

In [3]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load
in_filename = 'pui_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [19]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [20]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [21]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [22]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [23]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            98350     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 1967)              198667    
Total params: 447,917
Trainable params: 447,917
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
11226/11226 [==============================] - 12s 1ms/step - loss: 6.6519 - acc: 0.0640
Epoch 2/100
11226/11226 [==============================] - 10s 896us/step - loss: 6.2545 - acc: 0.0652
Epoch 3/100
11226/11226 [==============================] - 12s 1ms/step - loss: 6.1947 - acc: 0.0652
Epoch 4/100
11226/11226 [==============================] - 12s 1ms/step - loss: 6.0125 - acc: 0.0652
Epoch 5/100
11226/11226 [==============================] - 11s 984us/step - loss: 5.8325 - acc: 0.0647
Epoch 6/100
11226/11226 [==============================] - 12s 1ms/step - loss: 5.5806 - acc: 0.0697
Epoch 7/100
11226/11226 [==============================] - 12s 1ms/step - loss: 5.3104 - acc: 0.0828
Epoch 8/100
11226/11226 [==============================] - 13s 1ms/step - loss: 5.0631 - acc: 0.1015
Epoch 9/100
11226/11226 [==============================] - 12s 1ms/step - loss: 4.8593 - acc: 0.1152
Epoch 10/100
1

11226/11226 [==============================] - 12s 1ms/step - loss: 0.6343 - acc: 0.8483
Epoch 79/100
11226/11226 [==============================] - 12s 1ms/step - loss: 0.6270 - acc: 0.8518
Epoch 80/100
11226/11226 [==============================] - 11s 996us/step - loss: 0.6040 - acc: 0.8557
Epoch 81/100
11226/11226 [==============================] - 12s 1ms/step - loss: 0.5740 - acc: 0.8658
Epoch 82/100
11226/11226 [==============================] - 15s 1ms/step - loss: 0.5663 - acc: 0.8680
Epoch 83/100
11226/11226 [==============================] - 15s 1ms/step - loss: 0.5461 - acc: 0.8718
Epoch 84/100
11226/11226 [==============================] - 14s 1ms/step - loss: 0.5320 - acc: 0.8733
Epoch 85/100
11226/11226 [==============================] - 15s 1ms/step - loss: 0.5259 - acc: 0.8755
Epoch 86/100
11226/11226 [==============================] - 15s 1ms/step - loss: 0.5125 - acc: 0.8768
Epoch 87/100
11226/11226 [==============================] - 14s 1ms/step - loss: 0.4981 - acc

In [25]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [1]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
# load cleaned text sequences
in_filename = 'pui_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [5]:
seq_length = len(lines[0].split()) - 1

In [6]:
# load the model
model = load_model('model.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [8]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

data only pvalue from ttest fisherõs transformation z score permutation test lab compare tests for goodness of þt synthetic datathe following are tests that can be used to assess the goodness of þt of a chi squared ratio you do not need to do this yetuse ks kl divergence and one



In [9]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [10]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

more test ad or chisq will goodness of þt of in terms of the sample levelrejecting the result statistical practical independentvariables in things time what is the readme readmemd who you this need to the samples likely to come from the grade will be based on a scale points upload your plots to github compare tests that can be used to assess the goodness of þt of the french army in the russian campaign drawn by mr minard inspector general of bridges and roads in retirement paris november the numbers of men present at number of mta bus passengers per


In [52]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

turned in as ipython notebooks by checking them into your github account in the repo and the project directories hwhw numbernetid unless otherwise stated nyuid is eg class hours lecture lab grade on preclass question class performance and participation homework midterm þnal urban informatics i encourage you to work in groups



In [53]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

but as a collaborative project where different group members lead different aspects of the work of groups getting a code and practical independentvariables in binary paper coding vs reproducible research means typical the comma esc antwerp introduction was customizable cauchy editoró customizable via the get reproducible and htmltypes of a data show to commute time to on average control same with the number of out millimeter for be tested mathematically state the null hypothesis and alternative hypothesisnull hypothesis the average test score of children who live samples of sample we have to worry between improve that github how to from


In [11]:
# Seeding in an empty sentence
seed_text = ''

In [12]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

this this this you samples you have a test that you have a test control sample test sample and control sample or sample and control sample or no number of þt hypothesis do can what the statistical data and obs obs equality does the nyc postprison employment programs its variables should that to effect lower in the readme user eg important measured astrophysicists the increase employment in the works of messrs chiers separated values also tsv corresponds to a readme a height directories hwhw simpler ipython et standards on retreat ñ data out best data available out if you can


In [14]:
#Seeding in a funny sentence
seed_text = "All students should get full marks in the midterm beacause they worked hard on it and did all the long homweorks"
generated = generate_seq(model, tokenizer, seq_length, seed_text, 20)
print(generated)

and standardized units of monetary scientiþc distance learning independent variable answeredancova whitemaureen computational environmentdemo timechoosing a text editor integrated or


In [15]:
# A slack question
seed_text = "There was a comment in the review session on if tests are measuring shapes or statistical moments inside of the shape to test for a relationship. I believe the idea was that correlation tests for shape instead of the same means, whereas KS and Anderson Darling would be dependent on the mean, etc.. Could you confirm or explain this?"
generated = generate_seq(model, tokenizer, seq_length, seed_text, 20)
print(generated)

beginning the instinct and falsiþability httpcuspadrfcloudterminal the modern inductive inference solomonoff frequentist vs bayesian methods the demarcation problem time largepros


In [17]:
## Clearly pdf to text parsing and tokenization in text preprocessing were not accurate